In [6]:
import os 
import sys 
import logging 

logging.basicConfig(level=logging.INFO)

cur_path = os.path.abspath(".")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

from utils import remove_prefix, ddf 

load_dotenv()

True

In [7]:
sg = Subgrounds()
bs: Subgraph = sg.load_subgraph(os.environ['SUBGRAPH_URL'])

In [8]:
# df_ftokens = df_fert_tokens_raw.copy()

# nearest = alt.selection_single(
#     nearest=True, encodings=["x"], on="mouseover", empty="none"
# )
# brush = alt.selection_interval(encodings=['x'])

# base = alt.Chart(df_ftokens)
# x = alt.X("humidity:Q")
# y = alt.Y("fertilizer:Q")

# # Histogram of fertilizer purchased by season 
# histogram = base.mark_bar().encode(
#     x=x, y=y
# )

# # Vertical line to indicate nearest histogram bin 
# vline = base.mark_rule(color="grey").encode(
#     x=x, size=alt.value(1)
# ).transform_filter(
#     nearest
# )

# # Text label to indicate value nearest histogram bin 
# text = histogram.mark_text(align='left', dx=5, dy=-5).encode(
#     text=alt.condition(nearest, y, alt.value(' '))
# )

# # Time axis which drives scale of histogram 
# time_axis = base.mark_bar().encode(
#     x=x, y=y
# ).add_selection(
#     brush 
# ).resolve_scale(
#     'independent'
# ).properties(height=50)

# (
#     alt.layer(
#         histogram.add_selection(nearest), 
#         text, 
#         vline
#     ).transform_filter(
#         brush
#     ).properties(height=250) 
#     & time_axis 
# ) 

### Fertilizer Purchased by Humidity 

In [9]:
"""
For each season, we want to see deltaB and the amount of deltaB allocated 
to fertilizer maxNewFertilized
"""
def compute_deltaBeans(season, deltaB):
    season_multiplier = {
        szn: i/100 for i, szn in enumerate(range(6074, 6175))
    }
    return deltaB * season_multiplier.get(season, 1) / 1e6

def compute_maxNewFertilized(season, deltaB): 
    return max(0, compute_deltaBeans(season, deltaB)) / 3

bs.Season.deltaBeans = SyntheticField(
    compute_deltaBeans, SyntheticField.FLOAT, 
    [bs.Season.season, bs.Season.deltaB]
)
bs.Season.maxNewFertilized = SyntheticField(
    compute_maxNewFertilized, SyntheticField.FLOAT, 
    [bs.Season.season, bs.Season.deltaB]
)
seasons = bs.Query.seasons(
    first=10000, where={"season_gte": 6074}, orderBy="season", orderDirection="asc"
)
df_seasons_raw = sg.query_df([
    seasons.season, 
    seasons.deltaBeans, 
    seasons.maxNewFertilized, 
], pagination_strategy=ShallowStrategy)
df_seasons_raw = remove_prefix(df_seasons_raw, 'seasons_')
print(df_seasons_raw.maxNewFertilized.sum())
print(len(df_seasons_raw))
df_seasons_raw.head()


INFO:subgrounds:client.query: url = https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk, variables = {'first0': 900, 'skip0': 0}
query($first0: Int, $skip0: Int) {
  xea11476953cb434e: seasons(first: $first0, skip: $skip0, orderBy: season, orderDirection: asc, where: {season_gte: 6074}) {
    season
    deltaB
    id
  }
}


2183407.2825890235
204


,season,deltaBeans,maxNewFertilized
0,6074,0.000000,0.000000
1,6075,0.000000,0.000000
2,6076,13676.829341,4558.943114
3,6077,20556.793928,6852.264643
4,6078,27783.361943,9261.120648


In [10]:
# bs.FertilizerToken.sprouts = (
#     bs.FertilizerToken.supply * (bs.FertilizerToken.humidity + 100) / 100
# ) 
bs.FertilizerToken.fert = bs.FertilizerToken.supply 
bs.FertilizerToken.endBpf = SyntheticField(
  lambda _id: float(_id) / 1e6, 
  SyntheticField.FLOAT,
  bs.FertilizerToken.id, 
)
fert_tokens = bs.Query.fertilizerTokens(
    first=10000, 
    orderBy="humidity", 
    orderDirection="desc"
)
df_fert_tokens_raw = sg.query_df(
    [
        fert_tokens.season, 
        fert_tokens.fert, 
        fert_tokens.humidity, 
        fert_tokens.startBpf, 
        fert_tokens.endBpf, 
    ],
    pagination_strategy=ShallowStrategy
)
df_fert_tokens_raw = remove_prefix(df_fert_tokens_raw, "fertilizerTokens_")
df_fert_tokens_raw['startBpf'] /= 1e6
print(len(df_fert_tokens_raw))
df_fert_tokens_raw.head(10)


INFO:subgrounds:client.query: url = https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk, variables = {'first0': 900, 'skip0': 0}
query($first0: Int, $skip0: Int) {
  x9a5b0bd117500c08: fertilizerTokens(first: $first0, skip: $skip0, orderBy: humidity, orderDirection: desc, where: {}) {
    season
    supply
    humidity
    startBpf
    id
  }
}


79


,season,fert,humidity,startBpf,endBpf
0,6074,14364122,500.0,0.000000,6.000000
1,6074,70842,250.0,0.000000,3.500000
2,6075,3032,249.5,0.000000,3.495000
3,6076,31160,249.0,0.000157,3.490157
4,6077,11003,248.5,0.000472,3.485472
5,6078,4383,248.0,0.000951,3.480951
6,6079,15753,247.5,0.001597,3.476597
7,6080,15388,247.0,0.002520,3.472520
8,6081,100,246.5,0.003691,3.468691
9,6082,3095,246.0,0.005087,3.465087


### Experiment  

In [11]:
df_seasons_raw.head() 

,season,deltaBeans,maxNewFertilized
0,6074,0.000000,0.000000
1,6075,0.000000,0.000000
2,6076,13676.829341,4558.943114
3,6077,20556.793928,6852.264643
4,6078,27783.361943,9261.120648


In [12]:
df_fert_tokens_raw.head()

,season,fert,humidity,startBpf,endBpf
0,6074,14364122,500.0,0.000000,6.000000
1,6074,70842,250.0,0.000000,3.500000
2,6075,3032,249.5,0.000000,3.495000
3,6076,31160,249.0,0.000157,3.490157
4,6077,11003,248.5,0.000472,3.485472


In [17]:
# compute fertilizer active 
df = df_seasons_raw.merge(df_fert_tokens_raw, how="left", on="season")
assert len(df) == len(df)
df['unfertilized_index'] = ((df.endBpf - df.startBpf) * df.fert).fillna(0).cumsum() # validated 
# df['fert_active'] = df.fert.shift(1).fillna(0).cumsum()
# df['newBpf'] = (df.maxNewFertilized / df.fert_active).fillna(0) / 100
# df['fertilized_index'] = (df.fert_active * df.newBpf).cumsum()



# df['oldTotalBpf'] = df.newBpf.shift(1).fillna(0).cumsum()
# df['newTotalBpf'] = df.oldTotalBpf + df.newBpf
# TODO: fix stuff below here 
# df['oldTotalBpf'] = 
# df['bpf_i_test'] = (df.humidity + 100) / 100
# df['newBpf'] = (df.maxNewFertilized / df.fertilizer_active).fillna(0) / 100
# df['new_fertilized'] = df.fertilizer_active * df.newBpf
# df['fertilized_index'] = df['new_fertilized'].fillna(0).cumsum()
# df['cumulativeBpf'] = df.newBpf.cumsum()
# df.tail(50)

"""
season: The current season 
deltaBeans: The number of beans minted by beanstalk in the current season 
maxNewFertilized: The number of beans awarded to fertilizer in the current season's sunrise call 
"""
print(df.maxNewFertilized.sum())
print(df['unfertilized_index'].values.tolist()[-1])
ddf(df)

2183407.2825890235
95802657.03499998


,season,deltaBeans,maxNewFertilized,fert,humidity,startBpf,endBpf,unfertilized_index
0,6074,0.0,0.0,14364122.0,500.0,0.0,6.0,86184732.0
1,6074,0.0,0.0,70842.0,250.0,0.0,3.5,86432679.0
2,6075,0.0,0.0,3032.0,249.5,0.0,3.495,86443275.84
3,6076,13676.829,4558.943,31160.0,249.0,0.0,3.49,86552024.24
4,6077,20556.794,6852.265,11003.0,248.5,0.0,3.485,86590369.695
5,6078,27783.362,9261.121,4383.0,248.0,0.001,3.481,86605622.535
6,6079,35098.885,11699.628,15753.0,247.5,0.002,3.477,86660364.21
7,6080,48194.45,16064.817,15388.0,247.0,0.003,3.473,86713760.57
8,6081,59495.267,19831.756,100.0,246.5,0.004,3.469,86714107.07
9,6082,69514.434,23171.478,3095.0,246.0,0.005,3.465,86724815.77
